In [0]:
query = """
SELECT 
row_number() OVER(order by cc.customer_id) as customer_key,
cc.customer_id, cc.customer_number, cc.first_name, cc.last_name, ec.birth_date as birth_date ,
ecl.country,cc.marital_status, 
CASE 
WHEN
cc.gender<>'Unknown' THEN cc.gender
ELSE coalesce(ec.gender,'Unknown')
END as gender, 
cc.created_date
FROM silver.crm_customers as cc
LEFT JOIN silver.erp_customers as ec
ON cc.customer_number = ec.customer_number
LEFT JOIN silver.erp_customer_location as ecl
ON cc.customer_number = ecl.customer_number
"""
df = spark.sql(query)
df.limit(10).display()


### Check for NULLs

In [0]:
import pyspark.sql.functions as F
from pyspark.sql.types import StringType, DateType , IntegerType
from pyspark.sql.functions import trim, col

In [0]:
null_stats = (
    df.select([
        F.count(F.when(F.col(c).isNull(), c)).alias(c)
        for c in df.columns
    ])
)

display(null_stats)

Check for duplicates

In [0]:
df.groupBy("customer_id").count().filter(col("count") > 1).display()


# Writing the Gold Table

In [0]:
df.write.mode("overwrite").format("delta").saveAsTable("workspace.gold.dim_customers")

# Sanity checks

In [0]:
%sql

SELECT * FROM workspace.gold.dim_customers LIMIT 10
